In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import random
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model

print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

def seed_tensorflow(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

2024-07-29 13:05:05.059077: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-29 13:05:05.090907: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 13:05:05.090924: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 13:05:05.091901: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-29 13:05:05.097849: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-07-29 13:05:06.521851: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:c3:00.0, compute capability: 8.9
2024-07-29 13:05:06.528272: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:c3:00.0, compute capability: 8.9


In [2]:
# binary_train_label
# short_play
# effective_play
# long_play
# complete_play
select_label = 'short_play'

# base_model = "./model/kuairand_backbone/mlp_{}.h5".format(select_label)
base_model = "./model/wechat_base/autoint_{}.h5".format(select_label)

In [3]:
para = {'data_path':'./wechat_input/',
        'embedding_dim':64,
        'seed':0,
        'lr':1e-5,
        'batch_size':512,
        'epochs':64,
        'verbose':1,
        'callback':{
                     'monitor':'val_auc',
                     'patience':10,
                     'base_model':base_model
                     },
        'mlp_dims':[256,128,64],
        'mlp_act':'relu',
        'mlp_dps':[.5,.5,.5],
       }

In [4]:
action = pd.read_csv(para['data_path']+"action.csv")
feed_emb1 = np.load(para['data_path']+"feedid_emb_64.npy")
feed_emb2 = np.load(para['data_path']+"embeddings_array_wechat.npy")

In [5]:
action

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,...,videoplayseconds,pcr,duration_level,threshold,binary_train_label,short_play,effective_play,long_play,complete_play,test_label
0,0,63138,1,1,0,0,0,250,1533,0,...,16,0.015625,1,0.648438,0,1,0,0,0,0
1,0,43011,1,1,0,0,0,750,1302,0,...,31,0.024194,2,0.625000,0,1,0,0,0,0
2,0,23367,1,1,0,0,0,250,800,0,...,12,0.020833,0,0.656250,0,1,0,0,0,0
3,0,25886,1,1,0,0,0,0,1496,0,...,16,0.000000,1,0.648438,0,1,0,0,0,0
4,0,983,1,1,0,0,0,250,976,0,...,19,0.013158,1,0.648438,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7210182,19999,62197,12,2,0,0,0,6369,6566,0,...,29,0.219621,2,0.625000,0,1,0,0,0,0
7210183,19999,44479,12,2,0,0,0,13212,13708,0,...,11,1.000000,0,0.656250,1,0,1,1,1,0
7210184,19999,43644,12,2,0,0,0,0,32215,0,...,29,0.000000,2,0.625000,0,1,0,0,0,0
7210185,19999,40090,12,2,0,0,0,0,5618,0,...,12,0.000000,0,0.656250,0,1,0,0,0,0


In [6]:
action = action.sort_values(by=['date_'])

train= action.groupby('userid').apply(lambda x: x[:int(len(x)*0.6)]).reset_index(drop=True)
valid= action.groupby('userid').apply(lambda x: x[int(len(x)*0.6):int(len(x)*0.8)]).reset_index(drop=True)
test= action.groupby('userid').apply(lambda x: x[int(len(x)*0.8):]).reset_index(drop=True)

In [7]:
sparse_feats = ['userid','device','feedid','authorid','duration_level','feedid','feedid',select_label]

In [8]:
train_sparse_x = [train[f].values for f in sparse_feats]
val_sparse_x = [valid[f].values for f in sparse_feats]
test_sparse_x = [test[f].values for f in sparse_feats]

train_label = [train[select_label].values]
val_label = [valid[select_label].values]
test_label = [test[select_label].values]


In [9]:
train_sparse_x

[array([    0,     0,     0, ..., 19999, 19999, 19999]),
 array([1, 1, 1, ..., 2, 2, 2]),
 array([63138, 60079, 72045, ..., 76447, 73900,  3543]),
 array([ 1412,  8137, 11063, ..., 17608,  4653, 14210]),
 array([1, 0, 1, ..., 3, 4, 3]),
 array([63138, 60079, 72045, ..., 76447, 73900,  3543]),
 array([63138, 60079, 72045, ..., 76447, 73900,  3543]),
 array([1, 1, 1, ..., 1, 0, 1])]

In [10]:
seed_tensorflow(seed=para['seed'])
features = ['userid','device','feedid','authorid','feedid','duration_level','feedid',select_label]

# # short_play binary_train_label effective_play long_play complete_play
def get_input(df,is_test=False):
    X = []
    for f in features:
        X.append(df[f].values.reshape(-1,1))
    y = [df[select_label].values.reshape(-1,1)]
    return X,y

X_train,y_train = get_input(train,is_test=False)
X_valid,y_valid = get_input(valid,is_test=False)
X_test,y_test = get_input(test,is_test=True)

2024-07-29 13:05:32.805547: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:c3:00.0, compute capability: 8.9


In [11]:
sparse_inputs = []
'userid','device','feedid','authorid',
'feedid','duration_level','',select_label
input0 = tf.keras.layers.Input([1], name='userid',dtype=tf.int32)
input1 = tf.keras.layers.Input([1], name='device',dtype=tf.int32)
input2 = tf.keras.layers.Input([1], name='feedid',dtype=tf.int32)
input3 = tf.keras.layers.Input([1], name='authorid',dtype=tf.int32)
input4 = tf.keras.layers.Input([1], name='duration_level',dtype=tf.int32)
input5 = tf.keras.layers.Input([1], name='feedid1',dtype=tf.int32)
input6 = tf.keras.layers.Input([1], name='feedid2',dtype=tf.int32)
input7 = tf.keras.layers.Input([1], name=select_label,dtype=tf.int32)

sparse_inputs.append(input0)
sparse_inputs.append(input1)
sparse_inputs.append(input2)

sparse_inputs.append(input3)
sparse_inputs.append(input4)
sparse_inputs.append(input5)

sparse_inputs.append(input6)
sparse_inputs.append(input7)

In [12]:
sparse_kd_embed = []
for i, _input in enumerate(sparse_inputs):
    if i == 5:
        tf.keras.layers.Embedding(input_dim=int(feed_emb1.shape[0]),
                       output_dim=int(feed_emb1.shape[1]),
                       weights=[feed_emb1],
                       trainable=False,
                       name='pre_embedding1')
        sparse_kd_embed.append(_embed)
    elif i == 6:
        tf.keras.layers.Embedding(input_dim=int(feed_emb2.shape[0]),
                       output_dim=int(feed_emb2.shape[1]),
                       weights=[feed_emb2],
                       trainable=False,
                       name='pre_embedding2')
        sparse_kd_embed.append(_embed)
    elif i==7:
        continue
    else:
        f = sparse_feats[i]
        voc_size = action[f].nunique()
        _embed = tf.keras.layers.Embedding(voc_size+1, 64, embeddings_regularizer=tf.keras.regularizers.l2(0.5))(_input)
        sparse_kd_embed.append(_embed)
    


2024-07-29 13:05:32.868006: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:c3:00.0, compute capability: 8.9


In [13]:
sparse_kd_embed

[<KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_1')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_2')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_3')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_4')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_4')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_4')>]

In [14]:
input_embeds = sparse_kd_embed

In [15]:
embed_map = tf.keras.layers.Concatenate(axis=1)(input_embeds) 

In [16]:
embed_map

<KerasTensor: shape=(None, 7, 64) dtype=float32 (created by layer 'concatenate')>

In [17]:

def auto_interacting(embed_map, d=6, n_attention_head=2):
    """
    实现单层 AutoInt Interacting Layer
    @param embed_map: 输入的embedding feature map, (?, n_feats, n_dim)
    @param d: Q,K,V映射后的维度
    @param n_attention_head: multi-head attention的个数
    """
    assert len(embed_map.shape) == 3, "Input embedding feature map must be 3-D tensor."
    
    k = embed_map.shape[-1]
    
    # 存储多个self-attention的结果
    attention_heads = []
    W_Q = []
    W_K = []
    W_V = []
    
    # 1.构建多个attention
    for i in range(n_attention_head):
        # 初始化W_Q, W_K, W_V
        W_Q.append(tf.Variable(tf.random.truncated_normal(shape=(k, d)), name="query_"+str(i)))  # k, d
        W_K.append(tf.Variable(tf.random.truncated_normal(shape=(k, d)), name="key_"+str(i)))  # k, d
        W_V.append(tf.Variable(tf.random.truncated_normal(shape=(k, d)), name="value_"+str(i)))  # k, d
     
    for i in range(n_attention_head):
        # 映射到d维空间
        embed_q = tf.matmul(embed_map, W_Q[i])  # ?, 39, d
        embed_k = tf.matmul(embed_map, W_K[i])  # ?, 39, d
        embed_v = tf.matmul(embed_map, W_V[i])  # ?, 39, d
    
        # 计算attention
        energy = tf.matmul(embed_q, tf.transpose(embed_k, [0, 2, 1]))  # ?, 39, 39
        attention = tf.nn.softmax(energy)  # ?, 39, 39
    
        attention_output = tf.matmul(attention, embed_v)  # ?, 39, d
        attention_heads.append(attention_output)
    
    # 2.concat multi head
    multi_attention_output = tf.keras.layers.Concatenate(axis=-1)(attention_heads)  # ?, 39, n_attention_head*d
    
    # 3.ResNet
    w_res = tf.Variable(tf.random.truncated_normal(shape=(k, d*n_attention_head)), name="w_res_"+str(i))  # k, d*n_attention_head
    output = tf.keras.layers.Activation("relu")(multi_attention_output + tf.matmul(embed_map, w_res))  # ?, 39, d*n_attention_head)
    
    return output

In [18]:
def build_autoint(x0, n_layers):
    xl = x0
    for i in range(n_layers):
        xl = auto_interacting(xl, d=6, n_attention_head=2)
    
    return xl

In [19]:
autoint_layer = build_autoint(embed_map, 3)

In [20]:
autoint_layer = tf.keras.layers.Flatten()(autoint_layer)

In [21]:
autoint_layer

<KerasTensor: shape=(None, 84) dtype=float32 (created by layer 'flatten')>

In [22]:
flattened_embed_map = tf.keras.layers.Flatten()(embed_map)
flattened_embed_map

<KerasTensor: shape=(None, 448) dtype=float32 (created by layer 'flatten_1')>

In [23]:
vec = tf.keras.layers.Dense(256,activation = 'relu',
                name='mlp_dense0')(flattened_embed_map)

#vec = tf.keras.layers.Dropout(0.5)(vec)

vec = tf.keras.layers.Dense(128,
                            activation = 'relu',
                            name='mlp_dense1')(vec)

#vec = tf.keras.layers.Dropout(0.5)(vec)

instance = tf.keras.layers.Dense(64,
                            activation = 'relu',
                            name='instance')(vec)

In [24]:
final_instance = tf.keras.layers.Concatenate(axis=-1)([autoint_layer,instance]) 

In [25]:
final_instance

<KerasTensor: shape=(None, 148) dtype=float32 (created by layer 'concatenate_4')>

In [26]:
from tensorflow.keras import Model
base_model = Model(sparse_inputs, final_instance)
vec = base_model.outputs[0]

final_output = tf.keras.layers.Dense(1,"sigmoid")(vec)
model = Model(inputs=base_model.inputs, outputs=final_output)

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 userid (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 device (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 feedid (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 authorid (InputLayer)       [(None, 1)]                  0         []                            
                                                                                            

In [ ]:
seed_tensorflow(seed=para['seed'])
seed_tensorflow(seed=para['seed'])
# 20
adam=tf.keras.optimizers.Adam(learning_rate=para['lr'])
model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=['AUC'])



hist = model.fit(train_sparse_x,
                 train_label,
                 epochs=20,
                 batch_size=2048,
                 shuffle=True,
                 verbose=para['verbose'],
                 validation_data=(val_sparse_x,val_label))

2024-07-29 13:05:33.613490: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:c3:00.0, compute capability: 8.9
2024-07-29 13:05:33.615888: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22288 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:c3:00.0, compute capability: 8.9


Epoch 1/20


2024-07-29 13:05:37.142605: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f4e68cb7c90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-29 13:05:37.142624: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-07-29 13:05:37.146763: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-29 13:05:37.167306: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1722229537.240459  223741 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2109/2109 [==============================] - 23s 9ms/step - loss: 1999.5020 - auc: 0.5504 - val_loss: 908.5131 - val_auc: 0.6057
Epoch 2/20
2109/2109 [==============================] - 19s 9ms/step - loss: 431.3531 - auc: 0.5968 - val_loss: 133.5247 - val_auc: 0.6169
Epoch 3/20
2109/2109 [==============================] - 19s 9ms/step - loss: 45.0798 - auc: 0.6009 - val_loss: 5.1030 - val_auc: 0.6189
Epoch 4/20
2109/2109 [==============================] - 19s 9ms/step - loss: 1.4630 - auc: 0.6036 - val_loss: 0.6436 - val_auc: 0.6240
Epoch 5/20
 283/2109 [===>..........................] - ETA: 12s - loss: 0.6582 - auc: 0.6056

In [ ]:
model.save_weights(para['callback']['base_model'])

In [ ]:

# infer
"""
auc: 0.73412085
logloss: 0.57356197
"""


seed_tensorflow(seed=para['seed'])
model.load_weights(para['callback']['base_model'])
y_test_pred = model.predict(test_sparse_x,batch_size=10000)
auc_metric = tf.keras.metrics.AUC()
auc_metric.update_state(y_test[0], y_test_pred)
auc = auc_metric.result().numpy()

logloss_metric = tf.keras.metrics.BinaryCrossentropy()

logloss_metric.update_state(y_test[0], y_test_pred)
log_loss = logloss_metric.result().numpy()
print('auc:',auc)
print('logloss:',log_loss)